# Web Scraping

In [ ]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

I will be scraping the box office data from boxofficemojo.com using beautifulsoup, and collecting information I want such as gross and studio.

In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

yearBeginning = 2020 
finalDataList = []

for i in range(44): #getting movies years from 2020 - 1977
    try: 
        print("fetching data for year " + str(yearBeginning))
        page = 'https://www.boxofficemojo.com/year/' + str(yearBeginning) + '/?grossesOption=calendarGrosses'
        
        resp = requests.get(page)
        soup = BeautifulSoup(resp.text, 'lxml')
        table_data = [x.text for x in soup.select('tr td')]  # trial and error to get the exact positions
       
        distance = 11
        while distance <= len(table_data):
            table_data.insert(distance, str(yearBeginning))
            distance += 12

        temp_list = [table_data[i:i+12] for i in range(0, len(table_data[:-4]), 12)] # put every 5 values in a row

        yearBeginning = yearBeginning - 1 


        for movie in temp_list:
          finalDataList.append(movie)
   
    except Exception as e:
        break




In [ ]:
print("We have observations totaling " + str(len(finalDataList)) + " films.") 

In [ ]:
regex = '|'.join(['\$', ',', '\^'])


df = pd.DataFrame({
    'rank': [int(float(x[0])) for x in finalDataList],  # convert ranks to integers
    'title': [x[1] for x in finalDataList],  # get titles as is
    'studio': [x[9].strip('\n') for x in finalDataList],  # get studio names as is
    'total_gross': [x[7] for x in finalDataList],
    'gross': [x[5] for x in finalDataList],  
    'theaters': [x[6] for x in finalDataList],
    'release_date': [x[8] for x in finalDataList],  # get studio names as is
    'year': [x[11] for x in finalDataList],  
})
display(df)



# Cleaning Database

I would like to do some preliminary data exploration to check for errors or do some cleaning beforehand to later make my data vizs clearer

First, I want to check out what main studios there are

In [ ]:
display(df.studio.value_counts()[0:60])

I can see that Sony Pictures Entertainment (SPE) and Sony Pictures Classics are divisions of Sony Pictures, so I will make those two equal.

In [ ]:
df.loc[(df['studio'] == 'Sony Pictures Entertainment (SPE)'), ['studio']]= 'Sony Pictures'
df.loc[(df['studio'] == 'Sony Pictures Classics'), ['studio']]= 'Sony Pictures'


I will also drop the rows that have blank distributors as well as films during years that are re-releases. 

In [ ]:
df = df[(df['studio'] != '-')]
df = df[(df['title'].str.contains('release') == False )]


# Getting Genres

Unfortunately my scrapped data does not have genres for film and after multiple usages of boxofficemojo was faulty in searching for genre, I will now take the database and write a function that makes a scrap call for each film through IMBD. 

In [ ]:
df = pd.read_csv(r'moviesWithoutRelease.csv')
df = df[['rank', 'title', 'studio', 'total_gross', 'gross',
       'theaters', 'release_date', 'year', 'genres']]
df = df.reset_index(drop = True )
dfCurr = df.copy()


My computer was too slow and kept on crashing from scraping data or becoming unresponsive and so I will apply the function in smaller amounts slowly and save my database everytime, which is why my nStart and nEnd variables change

In [ ]:
titleFailedToGetGenre = []
def getGenreDirectly(x):
    title = x[1]
    title = title.replace(" ", "+")
    currentStr = ""
    page = 'https://www.imdb.com/search/title/?title=' + title
    try: 
        resp = requests.get(page)
        soup = BeautifulSoup(resp.text, 'lxml')
        currentStr = (soup.find("span", {"class": "genre"})).text.strip('\n')
        print(currentStr) #print the genre 
    except:
        print("error for " + page)
        titleFailedToGetGenre.append(title)
    return currentStr

nStart = 498
nEnd = 600
print("beginning.....")
dfCurr.iloc[nStart: nEnd]['genres'] = dfCurr.iloc[nStart: nEnd].apply(getGenreDirectly, axis=1)
print("done!")


In [ ]:
dfCurr.iloc[nStart - 2: nEnd  + 2]
display(dfCurr.iloc[nEnd - 10: nEnd  + 2])
dfCurr.to_csv(r'moviesWithGenres.csv', index = False)


# Getting Dates

In [ ]:
df = pd.read_csv(r'moviesWithGenres.csv')

df = df[['rank', 'title', 'studio', 'total_gross', 'gross', 'theaters',
       'release_date', 'year', 'genres']]




In [ ]:

import datetime

df['date'] =  (df['year'].astype(str))
df['date'] =  (df['release_date'].astype(str)) +  '-' + df['date']

#for no release dates, just keep year
df.loc[(df['release_date']  == '-'), ['date']]= '21-Jan-' + df['year'].astype(str)
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')



In [ ]:
from datetime import date, datetime


# Winter is the first day after New Year's week or weekend through the Thursday before the first Friday in March 01/05-03/05
# Spring is the first Friday in March through the Thursday before the first Friday in May.  03/06 - 05/05
# Summer is the first Friday in May through Labor Day weekend. 05/06- 09/5
# Fall is the day after Labor Day weekend through the Thrusday before the first Friday in November.   09/06-11/05
# Holiday Season is the first week in November till the first Thursday before the first Friday in January, unless that Friday is New Years then it's the next Thursday.
# 11/05-12/31

def getSeason(x):
  now = x[9]
  day = now.day
  month = now.month
  if month <= 3 : 
    if month == 3 and day >= 5: 
      return 'Spring'
    return 'Winter'
  elif month <= 5: 
    if month == 5 and day >= 5: 
      return 'Summer'
    return 'Spring'
  elif month <= 9: 
    if month == 9 and day >= 5: 
      return 'Fall'
    return 'Summer'
  elif month <= 11: 
    if month == 11 and day >= 5: 
      return 'Holiday Season'
    return 'Fall'
  else:
    return 'Holiday Season'

  return 'Fall'


df['season'] = df.apply(getSeason, axis=1)




In [ ]:
df

,rank,title,studio,total_gross,gross,theaters,release_date,year,genres,date,season
0,1,Bad Boys for Life,Sony Pictures,"$206,305,244","$204,417,855","3,775",17-Jan,2020,"Action, Comedy, Crime",2020-01-17,Winter
1,2,1917,Universal Pictures,"$159,227,644","$157,901,466","3,987",25-Dec,2020,"Drama, War",2020-12-25,Holiday Season
2,3,Sonic the Hedgehog,Paramount Pictures,"$146,066,470","$146,066,470","4,198",14-Feb,2020,"Action, Adventure, Comedy",2020-02-14,Winter
3,4,Jumanji: The Next Level,Sony Pictures,"$320,314,960","$124,736,710","4,227",13-Dec,2020,"Action, Adventure, Comedy",2020-12-13,Holiday Season
4,5,Star Wars: Episode IX - The Rise of Skywalker,Walt Disney Studios Motion Pictures,"$515,202,542","$124,496,308","4,406",20-Dec,2020,"Action, Adventure, Fantasy",2020-12-20,Holiday Season
...,...,...,...,...,...,...,...,...,...,...,...
15578,5,Exorcist II: The Heretic,Warner Bros.,"$30,749,142","$30,749,142",703,17-Jun,1977,NaN,1977-06-17,Summer
15579,6,The Turning Point,Twentieth Century Fox,"$25,933,445","$25,815,410",5,14-Nov,1977,NaN,1977-11-14,Holiday Season
15580,7,Looking for Mr. Goodbar,Paramount Pictures,"$22,512,655","$22,512,655",110,19-Oct,1977,NaN,1977-10-19,Fall
15581,8,Saturday Night Fever,Paramount Pictures,"$94,213,184","$18,234,852",726,16-Dec,1977,NaN,1977-12-16,Holiday Season


# Cleaning

In [ ]:
df['total_gross'] = df['total_gross'].str.replace('$', '')
df['gross'] = df['gross'].str.replace('$', '')
df['total_gross'] = df['total_gross'].str.replace(',', '')
df['gross'] = df['gross'].str.replace(',', '')
df['theaters'] = df['theaters'].str.replace(',', '')


In [ ]:

df['date_unix']  = pd.DatetimeIndex ( df.date ).astype ( np.int64 )/1000000


In [ ]:

df.to_csv(r'moviesData.csv', index = False)


I would like to get a list for select options of website so will calculate here an order of the top highest ranking distributors

In [ ]:
print("This is the method to get text")
# for x in listOfStudios: 
  # print('<option value="' + x + '">' + x + '</option>')


      

This is the method to get text


In [ ]:
df = pd.read_csv(r'moviesData.csv')


In [ ]:
sortedDF = df.groupby(['studio']).sum().sort_values('total_gross', ascending = False )

listOfStudios = sortedDF.index.values

I would also like to check the general quantiles of key points, such as the total_gross and gross

In [ ]:
# df.sort_values('date', ascending = True )

# df.quantile(0.5)
df.gross.quantile([.25, .50, .75])

# df.gross

df.columns

Index(['rank', 'title', 'studio', 'total_gross', 'gross', 'theaters',
       'release_date', 'year', 'genres', 'date', 'season', 'date_unix'],
      dtype='object')